In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'http://strainapi.evanbusse.com/80Bxz5h/strains/search/all'

In [5]:
response = requests.get(url)
dat = response.json()

In [6]:
#Creating Effects & Types DataFrame
names=list(dat.keys())
positive,negative,medical,posi,neg,med,eff,tye=[],[],[],[],[],[],[],[]
for n in names:
    tye.append(dat[n]['race'])
    eff.append(dat[n]['effects']['positive']+dat[n]['effects']['negative']+dat[n]['effects']['medical'])
    positive.append(dat[n]['effects']['positive'])
    posi.append(len(dat[n]['effects']['positive']))
    negative.append(dat[n]['effects']['negative'])
    neg.append(len(dat[n]['effects']['negative']))
    medical.append(dat[n]['effects']['medical'])
    med.append(len(dat[n]['effects']['medical']))
dic={'name':names,'type':tye,'effects':eff}
df=pd.DataFrame(dic)

In [7]:
df.head()

,name,type,effects
0,Afpak,hybrid,"[Relaxed, Hungry, Happy, Sleepy, Dizzy, Depres..."
1,African,sativa,"[Euphoric, Happy, Creative, Energetic, Talkati..."
2,Afternoon Delight,hybrid,"[Relaxed, Hungry, Euphoric, Uplifted, Tingly, ..."
3,Afwreck,hybrid,"[Relaxed, Happy, Creative, Uplifted, Sleepy, D..."
4,Agent Orange,hybrid,"[Relaxed, Euphoric, Happy, Energetic, Uplifted..."


In [8]:
#Cleaning up df, creating dummy columns for all effects
effects=[]
pos=list(eff)
for p in pos:
    for i in p:
        if i not in effects:
            effects.append(i)
for i in effects:
    title=str(i)
    title=[]
    for x in df.effects:
        if i in x:
            title.append(1)
        else:
            title.append(0)
    df[i]=title
#Get dummies for type (indica=0,sativa=1,hybrid=2)
#Engineer features for positive effect score, negative effect score, and medical effect score
df.drop(columns='effects',inplace=True)
df.type=df.type.map({'indica':0,'sativa':1,'hybrid':2})
lowers=[]
for n in df['name']:
    lowers.append(n.lower())
df['name']=lowers
df['positive']=posi
df['negative']=neg
df['medical']=med

In [9]:
df.head()

,name,type,Relaxed,Hungry,Happy,Sleepy,Dizzy,Depression,Insomnia,Pain,...,Eye Pressure,Anxious,Giggly,Aroused,Inflammation,Spasticity,Seizures,positive,negative,medical
0,afpak,2,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,4,1,5
1,african,1,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,5,1,6
2,afternoon delight,2,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,5,3,6
3,afwreck,2,1,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,5,4,6
4,agent orange,2,1,0,1,0,1,1,0,1,...,0,0,0,0,0,0,0,5,4,6


In [23]:
namelist, typelist, thclist, cbdlist, cbnlist, cbclist, thcvlist, cbglist = [],[],[],[],[],[],[],[]

responses = requests.get('https://www.potbotics.com/learn/strains')
soup = BeautifulSoup(responses.content,'html.parser')
names = soup.findAll('h4',{'class':'card-title'})
types = soup.findAll('h5',{'class':'type'})
thc = soup.findAll('p',{'class':'amount thc'})
cbd = soup.findAll('p',{'class':'amount cbd'})
cbn = soup.findAll('p',{'class':'amount cbn'})
cbc = soup.findAll('p',{'class':'amount cbc'})
thcv = soup.findAll('p',{'class':'amount thcv'})
cbg = soup.findAll('p',{'class':'amount cbg'})

for n, t, t2, c, c2, c3, t3, c4 in zip(names, types, thc, cbd, cbn, cbc, thcv, cbg):
    namelist.append(n.text)
    typelist.append(t.text)
    thclist.append(t2.text)
    cbdlist.append(c.text)
    cbnlist.append(c2.text)
    cbclist.append(c3.text)
    thcvlist.append(t3.text)
    cbglist.append(c4.text)

In [25]:
data_nums = {'name':namelist, 'type':typelist, 'thc':thclist, 'cbd':cbdlist, 'cbn':cbnlist, 'cbc':cbclist, 'thcv':thcvlist, 'cbg':cbglist}
num_df = pd.DataFrame(data_nums)
num_df.head()

,name,type,thc,cbd,cbn,cbc,thcv,cbg
0,3 Kings,SATIVA_DOMINANT,19.34,0.152,0,0.461,0.004,0
1,3x Crazy,INDICA_DOMINANT,16.47,0,0.069,0.168,0,0
2,Abusive OG,INDICA_DOMINANT,18.492,0.003,0.025,0.269,0.006,0
3,AC/DC,SATIVA_DOMINANT,0.876,8.93,0.03,0.138,0.11,0.142
4,Ace of Spades,INDICA_DOMINANT,14.53,0.003,0.189,0.052,0.003,0.095


In [27]:
num_df.sample(20)

,name,type,thc,cbd,cbn,cbc,thcv,cbg
72,Green Crack,SATIVA_DOMINANT,16.68,0.004,0.129,0.102,0,0.131
115,Mr. Nice,INDICA,9.17,0,0.077,0.064,0.004,0.114
15,Berry White,INDICA_DOMINANT,16.6,0,0.031,0.073,0.001,0.132
168,Tangerine Haze,SATIVA_DOMINANT,17.72,0.089,0.009,0.177,0,0.08
164,Super Lemon Haze,SATIVA_DOMINANT,4.53,8.78,0.014,0.139,0,0.051
167,Super Sour Diesel,SATIVA,16.16,0,0.061,0.133,0,0.167
137,Purple Gorilla,INDICA_DOMINANT,10.12,0,0.005,0.19,0.005,0
103,Liberty Haze,HYBRID,19.39,0,0.072,0.084,0.004,0.233
108,Maui Dream,SATIVA,16.12,0.184,0.211,0.487,0.006,0.212
25,Blue Ice,HYBRID,16.48,0.176,0.02,0.289,0,0.125


In [37]:
from matplotlib import pyplot as plt
import matplotlib_venn as venn
plt.figure(figsize=(10,5))
venn2(subsets=(len(num_df[num_df['type']==0]),len(num_df[num_df['type']==1]),len(num_df[num_df['type']==2])),set_labels=('Indica','Sativa'))
plt.savefig('Visualizations/VennDiagram_Indica_Sativa.png')

ModuleNotFoundError: No module named 'matplotlib_venn'